In [106]:
from drawpile import drawPile
import numpy as np
import ILP
from hand import hand
from board import board
from group import group
from tile import tile
'''
gather all of the tiles on the board and in your hand
count them up 0 to 2 for each tile, recorded in y array
y[i] array:
y[0:13] corresponds to Red tiles
y[13:26] corresponds to Blue tiles
y[26:39] corresponds to Yellow tiles
y[39:52] corresponds to Black tiles
y[52] corresponds to Joker

Ex: with hand R2 J0 Y8 K3 Y8 K6
y[1] = 1. y[33] = 2, y[41] = 1, y[44] = 1, y[52] = 1
all other y indices are 0

x[j] array:
as long as the total number of every possible group in the game
at each index is a 0,1, or 2
indicates how many times that group can be played based on hand+board

s[i][j] matrix:
each location i,j has 1 if tile i is in set j
0 if tile i not in set j

r[i] array:
tiles on your rack, same format as y[i]
y[i] <= r[i]
y[i] is tiles from rack that can be played, so if y[i] == r[i], you win

t[i] array:
tiles on the table, same format as y[i]

The hard part:
Maximize Sum of y[i] over all i∈{1,2...,53}
subject to: Sum of s[i][j]x[j] == t[i] + y[i]
            y[i] <= r[i]
            x[j] ∈ {0,1,2}
            y[j] ∈ {0,1,2}

source: doi:10.1093/comjnl/bxl033
'''
def checkLen(elem):
    return len(elem.group)

exampleDraw = drawPile()
exampleHand = hand(exampleDraw,1)

b = board()
b.addGroups([group([tile(4,'B'),tile(5,'B'),tile(6,'B')])])
b.addGroups([group([tile(4,'B'),tile(5,'B'),tile(6,'B')])])
b.addGroups([group([tile(9,'Y'),tile(10,'Y'),tile(11,'Y')])])
boardCopy = []
for item in b.board:
    boardCopy.extend(item.group)

exHand = exampleHand.hand + boardCopy
exHandStrings = [x.string for x in exHand]

exhaustiveList = ILP.generateSet(13,['R','B','K','Y'])
xarray = np.zeros(len(exhaustiveList))
for item in exhaustiveList:
    jokerCount = 0
    tileCounter = 0
    for el in item.group:
        if el.string in exHandStrings:
            tileCounter += 1
            if el.string == 'J0':
                exHandStrings.remove('J0')
                jokerCount += 1
    if tileCounter == len(item.group):
        xarray[exhaustiveList.index(item)] += 1
    exHandStrings.extend(jokerCount*['J0'])

In [107]:
print(exHandStrings)
print(xarray)
# print(exhaustiveList)
np.where(xarray==2)

['B4', 'B2', 'K11', 'R3', 'R6', 'B1', 'R7', 'B13', 'K8', 'R7', 'Y13', 'B9', 'R8', 'B4', 'B5', 'B6', 'B4', 'B5', 'B6', 'Y9', 'Y10', 'Y11', 'J0']
[0. 0. 0. ... 0. 0. 0.]


(array([], dtype=int64),)

In [109]:
groupsFromHand = np.where(xarray==1)
groupsFromHandDuplicates = np.where(xarray==2)
makeableGroups = 2*list(np.array(exhaustiveList)[groupsFromHand])
makeableGroups += 2*list(np.array(exhaustiveList)[groupsFromHandDuplicates])
makeableGroups.sort(key=checkLen,reverse=True)
for item in makeableGroups:
    print(*[x.string for x in item.group])


B1 B2 J0 B4 B5
B2 J0 B4 B5 B6
Y9 Y10 Y11 J0 Y13
B1 B2 J0 B4 B5
B2 J0 B4 B5 B6
Y9 Y10 Y11 J0 Y13
J0 R6 R7 R8
R6 R7 R8 J0
B1 B2 J0 B4
B2 J0 B4 B5
J0 B4 B5 B6
B4 B5 B6 J0
J0 Y9 Y10 Y11
Y9 Y10 Y11 J0
Y10 Y11 J0 Y13
J0 R6 R7 R8
R6 R7 R8 J0
B1 B2 J0 B4
B2 J0 B4 B5
J0 B4 B5 B6
B4 B5 B6 J0
J0 Y9 Y10 Y11
Y9 Y10 Y11 J0
Y10 Y11 J0 Y13
R6 R7 R8
B4 B5 B6
Y9 Y10 Y11
J0 R6 R7
J0 R7 R8
R6 J0 R8
R6 R7 J0
R7 R8 J0
B1 B2 J0
B2 J0 B4
J0 B4 B5
J0 B5 B6
B4 J0 B6
B4 B5 J0
B5 B6 J0
J0 Y9 Y10
J0 Y10 Y11
Y9 J0 Y11
Y9 Y10 J0
Y10 Y11 J0
Y11 J0 Y13
R6 B6 J0
R6 B6 J0
R8 J0 K8
R8 K8 J0
J0 B9 Y9
B9 J0 Y9
J0 K11 Y11
J0 K11 Y11
J0 B13 Y13
B13 J0 Y13
R6 R7 R8
B4 B5 B6
Y9 Y10 Y11
J0 R6 R7
J0 R7 R8
R6 J0 R8
R6 R7 J0
R7 R8 J0
B1 B2 J0
B2 J0 B4
J0 B4 B5
J0 B5 B6
B4 J0 B6
B4 B5 J0
B5 B6 J0
J0 Y9 Y10
J0 Y10 Y11
Y9 J0 Y11
Y9 Y10 J0
Y10 Y11 J0
Y11 J0 Y13
R6 B6 J0
R6 B6 J0
R8 J0 K8
R8 K8 J0
J0 B9 Y9
B9 J0 Y9
J0 K11 Y11
J0 K11 Y11
J0 B13 Y13
B13 J0 Y13


In [111]:
'''
start with 1 group, remove tiles from hand/board combo
'''
bestLen = 0
bestChoice = []
for i in range(len(makeableGroups)):
    # handCopy = exampleHand.hand[:]
    # handCopyStrings = [x.string for x in handCopy]
    boardCopy = []
    for item in b.board:
        boardCopy.extend(item.group)
    boardCopyStrings = [x.string for x in boardCopy]
    
    masterList = exHand[:]
    masterStrings = [x.string for x in masterList]

    useThese = []
    for item in makeableGroups[i:]:
        tempGroup = []
        for el in item.group:
            try:
                masterList.pop(masterStrings.index(el.string))
                masterStrings.remove(el.string)
            except:
                masterList.extend(tempGroup)
                masterStrings.extend([x.string for x in tempGroup])
                tempGroup = []
                break
            tempGroup.append(el)

        if tempGroup != []:
            useThese.append(tempGroup)
    
    newsThese = []
    for i in useThese:
        newsThese.extend(i)
    useTheseStrings = [x.string for x in newsThese]
    boardCheck = True
    for el in boardCopyStrings:
        if el in useTheseStrings:
            useTheseStrings.remove(el)
        else:
            boardCheck = False
            break
    
    if boardCheck and bestLen < len(newsThese):
        bestChoice = useThese[:]
        bestLen = len(newsThese)
        bestPlay = masterStrings[:]
             

for item in bestChoice:
    print([x.string for x in item])

print(bestPlay)

['B1', 'B2', 'J0', 'B4']
['R6', 'R7', 'R8']
['B4', 'B5', 'B6']
['Y9', 'Y10', 'Y11']
['B4', 'B5', 'B6']
['K11', 'R3', 'K8', 'Y13', 'R7', 'B9', 'B13']
